In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [4]:
from pydantic import BaseModel, Field

class Query(BaseModel):
    query: str = Field(..., description="The query to be executed")
    rationale: str = Field(..., description="The rationale for the query")

class QueryGenerationState(BaseModel):
    query_list: list[Query] = Field(
        default_factory=list,
        description="The list of queries to be executed",
    )

In [ ]:
from enum import Enum
from typing import List, Optional

from pydantic import BaseModel, Field


class StepType(str, Enum):
    RESEARCH = "research"
    PROCESSING = "processing"


class Step(BaseModel):
    need_search: bool = Field(..., description="Must be explicitly set for each step")
    title: str
    description: str = Field(..., description="Specify exactly what data to collect")
    step_type: StepType = Field(..., description="Indicates the nature of the step")
    query_list: Optional[list[Query]] = Field(
        default=None, description="The search query for the step"
    )
    execution_res: Optional[str] = Field(
        default=None, description="The Step execution result"
    )

class Plan(BaseModel):
    locale: str = Field(
        ..., description="e.g. 'en-US' or 'zh-CN', based on the user's language"
    )
    has_enough_context: bool
    thought: str
    title: str
    steps: List[Step] = Field(
        default_factory=list,
        description="Research & Processing steps to get more context",
    )

    class Config:
        json_schema_extra = {
            "examples": [
                {
                    "has_enough_context": False,
                    "thought": (
                        "To understand the current market trends in AI, we need to gather comprehensive information."
                    ),
                    "title": "AI Market Research Plan",
                    "steps": [
                        {
                            "need_search": True,
                            "title": "Current AI Market Analysis",
                            "description": (
                                "Collect data on market size, growth rates, major players, and investment trends in AI sector."
                            ),
                            "step_type": "research",
                        }
                    ],
                }
            ]
        }

In [ ]:
from langgraph.graph import MessagesState

class State(MessagesState):
    """State for the agent system, extends MessagesState with next field."""

    # Runtime Variables
    locale: str = "en-US"
    research_topic: str = ""
    model_selection: str = None
    observations: list[str] = []
    plan_iterations: int = 0
    current_plan: Plan | str = None
    auto_accepted_plan: bool = False

    # generate query
    initial_search_query_count: int = 3
    

In [5]:
from typing import Literal

# Define available LLM types
LLMType = Literal["gemini-2.5-flash", "gemini-2.5-pro", "claude-3.5-sonnet", "gpt-4o-mini", "deepseek-chat", "doubao-1.5-pro-32k"]

# Define agent-LLM mapping
AGENT_LLM_MAP: dict[str, LLMType] = {
    "coordinator": "gemini-2.5-flash",
    "planner": "gemini-2.5-flash",
    "researcher": "gpt-4o-mini",
    "coder": "gpt-4o-mini",
    "reporter": "gemini-2.5-flash",
    "podcast_script_writer": "doubao-1.5-pro-32k",
    "ppt_composer": "doubao-1.5-pro-32k",
    "prose_writer": "doubao-1.5-pro-32k",
    "prompt_enhancer": "doubao-1.5-pro-32k",
    "generate_query": "gpt-4o-mini",
}

In [6]:
import os
import yaml
from typing import Dict, Any

def replace_env_vars(value: str) -> str:
    """Replace environment variables in string values."""
    if not isinstance(value, str):
        return value
    if value.startswith("$"):
        env_var = value[1:]
        return os.getenv(env_var, env_var)
    return value

def process_dict(config: Dict[str, Any]) -> Dict[str, Any]:
    """Recursively process dictionary to replace environment variables."""
    if not config:
        return {}
    result = {}
    for key, value in config.items():
        if isinstance(value, dict):
            result[key] = process_dict(value)
        elif isinstance(value, str):
            result[key] = replace_env_vars(value)
        else:
            result[key] = value
    return result

_config_cache: Dict[str, Dict[str, Any]] = {}

def load_yaml_config(file_path: str) -> Dict[str, Any]:
    """Load and process YAML configuration file."""
    # 如果文件不存在，返回{}
    if not os.path.exists(file_path):
        return {}

    # 检查缓存中是否已存在配置
    if file_path in _config_cache:
        return _config_cache[file_path]

    # 如果缓存中不存在，则加载并处理配置
    with open(file_path, "r") as f:
        config = yaml.safe_load(f)
    processed_config = process_dict(config)

    # 将处理后的配置存入缓存
    _config_cache[file_path] = processed_config
    return processed_config


In [7]:
from pathlib import Path
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic
# Cache for LLM instances
_llm_cache: dict[LLMType, ChatOpenAI] = {}

def _get_config_file_path() -> str:
    """Get the path to the configuration file."""
    return str((Path(os.getcwd()) / "conf.yaml").resolve())

def _get_llm_type_config_keys() -> dict[str, str]:
    """Get mapping of LLM types to their configuration keys."""
    return {
        "gemini-2.5-flash": "GEMINI_2_5_FLASH",
        "gemini-2.5-pro": "GEMINI_2_5_PRO",
        "gpt-4o-mini": "GPT_4O_MINI",
        "doubao-1.5-pro-32k": "DOUBAO_1_5_PRO_32K",
        "claude-3.5-sonnet": "CLAUDE_3_5_SONNET",
        "deepseek-chat": "DEEPSEEK_CHAT",
    }

def _get_env_llm_conf(llm_type: str) -> Dict[str, Any]:
    """
    Get LLM configuration from environment variables.
    Environment variables should follow the format: {LLM_TYPE}__{KEY}
    e.g., BASIC_MODEL__api_key, BASIC_MODEL__base_url
    """
    prefix = f"{llm_type.upper()}_MODEL__"
    conf = {}
    for key, value in os.environ.items():
        if key.startswith(prefix):
            conf_key = key[len(prefix) :].lower()
            conf[conf_key] = value
    return conf

def _create_llm_use_conf(
    llm_type: LLMType, conf: Dict[str, Any]
) -> ChatOpenAI | ChatGoogleGenerativeAI | ChatAnthropic:
    """Create LLM instance using configuration."""
    llm_type_config_keys = _get_llm_type_config_keys()
    config_key = llm_type_config_keys.get(llm_type)

    if not config_key:
        raise ValueError(f"Unknown LLM type: {llm_type}")

    llm_conf = conf.get(config_key, {})
    if not isinstance(llm_conf, dict):
        raise ValueError(f"Invalid LLM configuration for {llm_type}: {llm_conf}")

    # Get configuration from environment variables
    env_conf = _get_env_llm_conf(llm_type)

    # Merge configurations, with environment variables taking precedence
    merged_conf = {**llm_conf, **env_conf}

    if not merged_conf:
        raise ValueError(f"No configuration found for LLM type: {llm_type}")

    # if llm_type == "reasoning":
    #     merged_conf["api_base"] = merged_conf.pop("base_url", None)
    model_name = merged_conf.get("model")
    if not model_name:
        raise ValueError(f"No model name configured for LLM type: {llm_type}")

    if "gemini" in model_name:
        return ChatGoogleGenerativeAI(**merged_conf)
    elif "claude" in model_name:
        return ChatAnthropic(**merged_conf)
    else:
        # Default to ChatOpenAI for OpenAI, Bytedance, and other compatible models.
        return ChatOpenAI(**merged_conf)

def get_llm_by_type(
    llm_type: LLMType,
) -> ChatOpenAI | ChatGoogleGenerativeAI | ChatAnthropic:
    """
    Get LLM instance by type. Returns cached instance if available.
    """
    if llm_type in _llm_cache:
        return _llm_cache[llm_type]

    conf = load_yaml_config(_get_config_file_path())
    llm = _create_llm_use_conf(llm_type, conf)
    _llm_cache[llm_type] = llm
    return llm

In [8]:
import os
from dataclasses import dataclass, fields
from typing import Any, Optional

from langchain_core.runnables import RunnableConfig



@dataclass(kw_only=True)
class Configuration:
    """The configurable fields."""


    max_plan_iterations: int = 1  # Maximum number of plan iterations
    max_step_num: int = 3  # Maximum number of steps in a plan
    max_search_results: int = 3  # Maximum number of search results
    mcp_settings: dict = None  # MCP settings, including dynamic loaded tools

    enable_deep_thinking: bool = False  # Whether to enable deep thinking
    number_of_initial_queries: int = 3  # Number of initial search queries to generate in sub-graph

    @classmethod
    def from_runnable_config(
        cls, config: Optional[RunnableConfig] = None
    ) -> "Configuration":
        """Create a Configuration instance from a RunnableConfig."""
        configurable = (
            config["configurable"] if config and "configurable" in config else {}
        )
        values: dict[str, Any] = {
            f.name: os.environ.get(f.name.upper(), configurable.get(f.name))
            for f in fields(cls)
            if f.init
        }
        return cls(**{k: v for k, v in values.items() if v})


In [9]:
import os
import dataclasses
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape
from langgraph.prebuilt.chat_agent_executor import AgentState

# Initialize Jinja2 environment
env = Environment(
    loader=FileSystemLoader('./prompts'),
    autoescape=select_autoescape(),
    trim_blocks=True,
    lstrip_blocks=True,
)
def apply_prompt_template(
    prompt_name: str, state: AgentState, configurable: Configuration = None
) -> list:
    """
    Apply template variables to a prompt template and return formatted messages.

    Args:
        prompt_name: Name of the prompt template to use
        state: Current agent state containing variables to substitute

    Returns:
        List of messages with the system prompt as the first message
    """
    # Convert state to dict for template rendering
    state_vars = {
        "CURRENT_TIME": datetime.now().strftime("%a %b %d %Y %H:%M:%S %z"),
        **state,
    }

    # Add configurable variables
    if configurable:
        state_vars.update(dataclasses.asdict(configurable))

    try:
        template = env.get_template(f"{prompt_name}.md")
        system_prompt = template.render(**state_vars)
        return [{"role": "system", "content": system_prompt}] + state.get("messages", [])
    except Exception as e:
        raise ValueError(f"Error applying template {prompt_name}: {e}")

In [10]:
class SearchQueryList(BaseModel):
    query: List[str] = Field(
        description="A list of search queries to be used for web research."
    )
    rationale: str = Field(
        description="A brief explanation of why these queries are relevant to the research topic."
    )

In [ ]:
import logging
from langchain_core.runnables import RunnableConfig
from langgraph.types import Command
from langchain_core.messages import HumanMessage

logger = logging.getLogger(__name__)

def generate_query(state: State, config: RunnableConfig):
    """LangGraph node that generates a search queries based on the User's question.

    Create an optimized search query for web research based on the User's question.

    Args:
        state: Current graph state containing the User's question
        config: Configuration for the runnable, including LLM provider settings

    Returns:
        Dictionary with state update, including search_query key containing the generated query
    """
    configurable = Configuration.from_runnable_config(config)

    # check for custom initial search query count
    if state.get("initial_search_query_count") is None:
        state["initial_search_query_count"] = configurable.number_of_initial_queries

    current_plan = state.get("current_plan")

    current_step = None
    completed_steps = []
    for step in current_plan.steps:
        if not step.execution_res:
            current_step = step
            break
        else:
            completed_steps.append(step)

    if not current_step:
        logger.warning("No unexecuted step found")
        return Command(goto="research_team")

    logger.info(f"Executing step: {current_step.title}")

    # Format completed steps information
    completed_steps_info = ""
    if completed_steps:
        completed_steps_info = "# Existing Research Findings\n\n"
        for i, step in enumerate(completed_steps):
            completed_steps_info += f"## Existing Finding {i + 1}: {step.title}\n\n"
            completed_steps_info += f"<finding>\n{step.execution_res}\n</finding>\n\n"



    messages = apply_prompt_template("generate_query", state, configurable)
    messages.append(
        HumanMessage(
            content=f"{completed_steps_info}# Current Task\n\n## Title\n\n{current_step.title}\n\n## Description\n\n{current_step.description}\n\n## Locale\n\n{state.get('locale', 'en-US')}"
        )
    )
    llm = (
        get_llm_by_type(AGENT_LLM_MAP["generate_query"])
        .with_structured_output(SearchQueryList)
    )

    response = llm.invoke(messages)
    state["query_list"] = response.query

    return {"query_list": response.query}

In [12]:
plan = Plan(
    locale="en-US",
    has_enough_context=False,
    thought="To understand the current market trends in AI, we need to gather comprehensive information.",
    title="AI Market Research Plan",
    steps=[
        Step(
            need_search=True,
            title="Current AI Market Analysis",
            description="Collect data on market size, growth rates, major players, and investment trends in AI sector.",
            step_type=StepType.RESEARCH
        )
    ]
)


In [13]:
# generate_query 调用测试
from langchain_core.runnables import RunnableConfig

# 构造 state
state = State(
     # Runtime Variables
    locale= "en-US",
    research_topic=  "",
    model_selection=  None,
    observations= [],
    plan_iterations=  0,
    current_plan= plan,
    auto_accepted_plan=  False,

    # generate query
    initial_search_query_count= 3
)

# 构造 config
config = RunnableConfig({})

# 调用 generate_query
result = generate_query(state, config)

# 打印结果
print(result)

{'query_list': ['AI market size and growth rates July 2025', 'Major players in the AI market July 2025', 'Current investment trends in the AI sector July 2025']}


In [ ]:
from langgraph.types import Send

def continue_to_generic_tool_executor(state: QueryGenerationState):
    """LangGraph node that sends the search queries to the web research node.

    This is used to spawn n number of web research nodes, one for each search query.
    """
    return [
        Send("generic_tool_executor", {"search_query": search_query, "id": int(idx)})
        for idx, search_query in enumerate(state["query_list"])
    ]

In [ ]:
class GenericToolState(BaseModel):
    
    )

In [ ]:
def generic_tool_executor(state: State, config: RunnableConfig
) -> GenericToolState: